In [ ]:
pip install -r requirements.txt

In [2]:
import os
import tiktoken
import numpy as np
import openai
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [ ]:

# Load environment variables
load_dotenv()

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = "https://testsopenaiajay.openai.azure.com/" #os.getenv('OPENAI_API_BASE')
openai.api_key = "ff5e3e2c133340da83c9e8dcb6d25bb9" #os.getenv("OPENAI_API_KEY")

# Define embedding model and encoding
EMBEDDING_MODEL = 'text-embedding-ada-002'
COMPLETION_MODEL = 'text-davinci-003'
encoding = tiktoken.get_encoding('cl100k_base')


In [4]:
df = pd.read_csv('./data/movies.csv')
print(df.shape)
df.head()

(500, 12)


,id,original_language,original_title,popularity,release_date,vote_average,vote_count,genre,overview,revenue,runtime,tagline
0,381284.0,en,Hidden Figures,49.802,2016-12-10,8.1,7310.0,"['Drama', 'History']","The untold story of Katherine G. Johnson, Doro...",2.306988e+08,127.0,"Meet the women you don't know, behind the miss..."
1,356334.0,en,Gridlocked,9.801,2016-06-14,5.8,130.0,['Action'],Former SWAT leader David Hendrix and hard-part...,0.000000e+00,114.0,Only one way out…
2,475557.0,en,Joker,116.462,2019-10-02,8.2,18970.0,"['Crime', 'Thriller', 'Drama']","During the 1980s, a failed stand-up comedian i...",1.074251e+09,122.0,Put on a happy face.
3,347847.0,en,The Sand,14.172,2015-08-28,5.1,157.0,['Horror'],Just when you thought it was safe to go back i...,0.000000e+00,84.0,This Beach is Killer
4,739542.0,en,America: The Motion Picture,98.542,2021-06-30,5.8,130.0,"['Action', 'Comedy', 'History', 'Animation', '...",A chainsaw-wielding George Washington teams wi...,0.000000e+00,98.0,This summer they're redrawing history.


In [ ]:
# add a new column to the dataframe where you put the token count of the review
df = df.assign(token_count=df['overview'].apply(lambda x: len(encoding.encode(x))))

# print the first 5 rows of the dataframe, then also the total number of tokens
total_tokens = df['token_count'].sum()

cost_for_embeddings = total_tokens / 1000 * 0.0004
print(f"Test would cost ${cost_for_embeddings} for embeddings")

In [ ]:
#@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def get_embedding(text) -> list[float]:
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=text, engine="embeddingada")["data"][0]["embedding"]

In [ ]:
df = df.assign(embedding=df['overview'].apply(lambda x: get_embedding(x)))
df.head()

In [ ]:
# Let's pick a movie that exists in df, keeping in mind we only have 500 movies in it!
movie = "Joker"

# get embedding for movie
e = df[df['original_title'] == movie]['embedding'].values[0]

# get cosine similarity between movie and all other movies and sort ascending
similarities = df['embedding'].apply(lambda x: cosine_similarity(x, e))

# get most similar movies
movies = df.assign(similarity=similarities).sort_values(by='similarity', ascending=False)[['original_title', 'similarity', 'overview']]
movies[0:6]

In [ ]:
df.to_csv('file1.csv')

df1 = pd.read_csv ('file1.csv')

In [ ]:

np.save('embeddings.npy', df)

In [3]:
embeddings = np.load('embeddings.npy', allow_pickle=True)

df1 = pd.DataFrame(embeddings,columns=['id','original_language','original_title','popularity','release_date','vote_average','vote_count','genre','overview','revenue','runtime','tagline','token_count','embedding'])
# Let's pick a movie that exists in df, keeping in mind we only have 500 movies in it!
movie = "The Italian Job"

# get embedding for movie
e = df1[df1['original_title'] == movie]['embedding'].values[0]

# get cosine similarity between movie and all other movies and sort ascending
similarities = df1['embedding'].apply(lambda x: cosine_similarity(x, e))

# get most similar movies
movies = df1.assign(similarity=similarities).sort_values(by='similarity', ascending=False)[['original_title', 'similarity', 'overview']]
movies[0:6]

,original_title,similarity,overview
145,The Italian Job,1.000000,Charlie Croker pulled off the crime of a lifet...
15,Wheelman,0.859101,A getaway driver for a bank robbery realizes h...
255,Asher,0.843402,Asher is a former Mossad agent turned gun for ...
70,Carnage Park,0.842300,"Part crime caper gone awry, part survival horr..."
75,Night Teeth,0.840967,A college student moonlighting as a chauffeur ...
210,To Catch a Thief,0.839867,An ex-thief is accused of enacting a new crime...
